# Taxi company car allocation


### Optimization Methods, MA06002, 2016

* Artem Korenev
* Andrei Kvasov
* Anton Marin
* Oleg Sudakov

It is common knowledge, that the financial efficiency, or, equivalently, profit of a taxi company depends on how many customers its drivers will be able to serve, and what resources should be spent to achieve such result. The company should also consider its reputation as a brand, which depends on, among other factors, on how fast its cars are able to pick up clients and to drive them to their destination. 

Both of these metrics greatly depend on how many cars a company should employ, and where they should be allocated geographically during the day.

In this problem, our team will examine historical data of Uber pickups in New York City, collected from April to September 2014.

### Problem formulation

The dataset contains the following information about Uber pickups in New York City: Date and time of the Uber pickup, the latitude of the Uber pickup, and the longitude of it. The pickup locations will be represented as points of a graph, where the weight of a given edge between vertices will be the distance between two points. We will be solving vehicle allocation problem, assuming that we know the orders' locations and times in advance.

Let's examine elements of the problem individually:

#### Taxi drivers

The total number of taxi drivers will be denoted as $N$. Each driver will start his working day at position $X_0^j$ at time $T_0^j$, where $j$ denotes the index of a taxi car.

We will assume, that the driver's shift has a following schedule:

1. The car $j$ waits for orders at a specific point $X_w^j$, starting from the time $T_w^j$;
2. The car drives to pick up the client $k$ at the same point at time $T_p^k$;
3. After picking up the client, car immediately drives to its destination $X_d$. It arrives at time $T_a^k$;
4. The car drops the client at the destination point and afterwards heads to the next pickup point $X_{w+1}^j$ to wait for the order. It starts waiting for the next client at time $T_{w+1}^j$ (if the car waits for the next client at the drop-off point of the previous order, $T_{w+1}^j = T_a^k$.

Obviously, $T_a^k = T_p^k + time(X_p, X_d)$, and $T_{w+1}^j = T_a^k + time(X_d, X_{w+1}^j)$, where the $time(A, B)$ denotes the time needed to get from point $A$ to point $B$. $T_p^k$ is given by the dataset.

We will introduce two penalties for each car:

* $c_{d}$ - downtime cost per unit of time, induced on each taxi car when it is stationary.
* $c_{f}$ - fuel cost per unit of time, induced on each taxi car when it is en route.

Therefore, if the car $j$ is assigned to order $k$, the penalty for this individual order equals to:

$$P_{j, k} = time(X_w^j, X_p)c_{f} + (T_p^k - T_w^j - time(X_w^j, X_p))c_{d} + time(X_p, X_d)c_{f} + time(X_d, X_{w+1}^j)c_{f}$$

This formula does not take into account the downtime at the end of the day. Given end of workshift time $T_{end}$, car index $j$ and the number of orders that this car completed during the day as $n_j$ the penalty for the downtime at the end of the day for this car is $$E_j = (T_{end} - T^j_{n_j})c_{d}$$

The penalty for maintaining individual car will be denoted as $c_{a}$ ($a$ stands for auto). Then, the additional penalty for maintaining $N$ cars on a given day equals to $Nc_a$

#### Clients

The total number of clients, or orders will be denoted as $C$. Individual order is represented by a tuple $(t^k, X_p, X_d)$, where $t^k$ denotes the minimum possible time for a pickup, $X_p$ the pickup point, and $X_d$ the destination point. 

A cost for a late pickup of the client will be denoted as $c_{t}$. Therefore, for individual order $k$ the penalty for the pickup delay equals to $D_k = (T_p^k - t^k)c_{t}$.

#### Total penalty

To sum up, we will aim to minimize taxi car downtime, pickup delays and fuel costs. Thus, for a given formulation of a problem our goal is:

$$\Sigma_{j, k = 1}^{N, C}P_{j, k} + \Sigma_{j=1}^N E_j + Nc_a + \Sigma_{k=1}^C D_k \rightarrow min$$








# Chosen Methods

## 1. Greedy Algorithms

One of the most straightforward approaches to the problem are greedy algorithms. We can perform the greedy steps in order to make decisions upon pick ups of clients. However, it can be clearly seen almost instantly that applying such algorithms will not produce an optimal solution. However, applying such technique we can reach a sufficient approximation.

## 2. Dynamic Programming

## 3. Linear Programming

## 4. Integer Programming

## 5. Flow Problem